In [ ]:
# Notebook 01: Synthetic Data Factory
import google.generativeai as genai
import json
import os
import time
from dotenv import load_dotenv

# Configurar Google Gemini
load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel("models/gemini-2.5-flash")

# 1. cargar frases de "Alicia en el País de las Maravillas" en chunks_nltk.txt
with open("../data/chunks_nltk.txt", "r", encoding="utf-8") as f:
    texto_alicia = f.read().split("\n\n")

In [ ]:
# Función para generar el par sintético
def generar_par_sintetico(frase_original):
    prompt = f"""
    Eres un experto lingüista. Tu tarea es simplificar esta frase de Lewis Carroll 
    a un lenguaje extremadamente plano, aburrido y moderno, manteniendo el sentido:
    "{frase_original}"
    Respuesta corta, solo la frase simplificada.
    """
    response = model.generate_content(prompt)
    return response.text.strip()

In [5]:
# 2. Procesamiento con guardado incremental y recuperación de estado
total_frases = len(texto_alicia)
output_file = "../data/synthetic_alicia.jsonl"
checkpoint_file = "../data/checkpoint.txt"

# Leer checkpoint si existe (última posición procesada)
start_index = 0
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r") as cf:
        start_index = int(cf.read().strip())
    print(f"Reanudando desde la frase {start_index + 1}")

contador = start_index

# Abrir archivo en modo append si continuamos, o write si empezamos de cero
mode = "a" if start_index > 0 else "w"
try:
    with open(output_file, mode, encoding="utf-8") as f:
        for i, frase in enumerate(texto_alicia[start_index:], start=start_index + 1):
            try:
                input_plano = generar_par_sintetico(frase)
                entry = {
                    "input": input_plano,
                    "output": frase
                }
                # Guardar inmediatamente cada entrada
                f.write(json.dumps(entry, ensure_ascii=False) + "\n")
                f.flush()  # Forzar escritura al disco
                
                # Actualizar checkpoint
                with open(checkpoint_file, "w") as cf:
                    cf.write(str(i))
                
                contador = i
                print(f"Procesado y guardado {i}/{total_frases} frases")
                
                # Delay para evitar superar el rate limit (3 segundos entre requests)
                if i < total_frases:
                    time.sleep(3)
                    
            except Exception as e:
                error_msg = str(e)
                # Detectar error de cuota
                if "quota" in error_msg.lower() or "rate" in error_msg.lower() or "429" in error_msg:
                    print(f"\n❌ Error de cuota detectado en frase {i}: {e}")
                    print(f"Progreso guardado. Ejecuta de nuevo para continuar desde la frase {i}.")
                    raise  # Re-lanzar para salir del script
                else:
                    print(f"⚠️ Error en frase {i}: {e} - Continuando...")
                    continue
    
    # Si llegamos aquí, terminamos exitosamente
    print(f"\n✅ Total procesado y guardado: {contador} frases")
    # Eliminar checkpoint al finalizar
    if os.path.exists(checkpoint_file):
        os.remove(checkpoint_file)
        print("Checkpoint eliminado - proceso completado.")
        
except Exception as e:
    print(f"\n💾 Proceso interrumpido. Última frase procesada: {contador}")
    print(f"Para continuar, ejecuta esta celda de nuevo.")


❌ Error de cuota detectado en frase 1: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 35.510824599s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 20
}
, retry_delay {
  seconds: 35
}
]
Progreso guardado. Ejecuta de nuevo para continuar desde la frase 1.
